# 유사성 분석을 테스트하기 위한 노트북입니다.

## 준비

쓸모없는 칼럼을 제거한 전용 데이터를 만듭니다.

이 작업은 엑셀이 더 편하므로 엑셀에서 합니다.

1. 특수 npc를 전부 날립니다. 특수 npc들은 속성이 너무 부족합니다.
2. `special` 열은 지웁니다.
3. `image_photo`, `image_house` 열도 지웁니다. `image_icon`은 혹시 모르니 남겨 두겠습니다.
4. `gender` 열을 지우고 `gender_asia`를 `gender`로 이름을 바꿉니다.
5. 마찬가지로 `personality_subtype`, `catchphrase`, `favorite_song`, `favorite_saying`, `tier`, `rank`도 사용자에게 질문할 수 없는 항목이므로 버립니다.
6. 나머지는 좀 더 생각이 필요하거나 코드로 해야 편한 작업들인 것 같습니다.
7. 빈 항목이 없도록 주의